In [1]:
import os
# import sys
# sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID')

In [2]:
from readers.bilicough_reader import BiliCoughReader
bcr = BiliCoughReader(ROOT="G:/DATAS-Medical/BILIBILICOUGH/")

In [ ]:
sample_list, label_list = bcr.get_sample_label_list()
print("number of sample:{}, positive label:{}".format(len(label_list), sum(label_list)))
print("sample rate:{}, data length:{}".format(bcr.sr, bcr.data_length))

In [1]:
from readers.neucough_reader import NEUCoughReader
ncr = NEUCoughReader()
print("wave root:", ncr.ROOT)
print("meta path:", ncr.metapath)

wave root: F:/DATAS/NEUCOUGHDATA_FULL/
meta path: F:/DATAS/NEUCOUGHDATA_COUGHSINGLE/neucough_metainfo.txt


In [2]:
sample_list, label_list = ncr.get_sample_label_list()

100%|████████████████████████████████████████████████████████████████████████████████| 321/321 [00:26<00:00, 12.09it/s]


In [4]:
print("sample rate:{}, data length:{}".format(ncr.sr, ncr.data_length))
print("number of sample:{}".format(len(label_list)))
print("length of data:", len(sample_list[0]))

sample rate:22050, data length:22050
number of sample:321
length of data: 22050


In [6]:
from readers.coughvid_reader import CoughVIDReader
cvr = CoughVIDReader()
print("wave root:", ncr.ROOT)
print("meta path:", ncr.metapath)

wave root: F:/DATAS/NEUCOUGHDATA_FULL/
meta path: F:/DATAS/NEUCOUGHDATA_COUGHSINGLE/neucough_metainfo.txt


In [7]:
sample_list, label_list = cvr.get_sample_label_list()

100%|████████████████████████████████████████████████████████████████████████████| 2850/2850 [00:00<00:00, 3017.75it/s]


In [10]:
print("sample rate:{}, data length:{}".format(cvr.sr, cvr.data_length))
print("number of sample:{}".format(len(label_list)))
print("length of data:", len(sample_list[0]))

sample rate:None, data length:None
number of sample:2850
length of data: 22050


In [20]:
import numpy as np
arr = np.array([1,2,3,4,5,6])
L = np.zeros(20)
resi = len(L) - len(arr)
L[:len(arr)] = arr
L[-len(arr):] = arr[-resi:][::-1]
L

array([1., 2., 3., 4., 5., 6., 0., 0., 0., 0., 0., 0., 0., 0., 6., 5., 4.,
       3., 2., 1.])

# 未经细心清洗的CoughVID数据集

In [1]:
import numpy as np
def CoughVID_Lists(filename="F:/DATAS/COUGHVID-public_dataset_v3/annotation.csv", isdemo=False):
    path_list = []
    label_list = []
    with open(filename, 'r') as fin:
        fin.readline()
        line = fin.readline()
        ind = 0
        while line:
            parts = line.split(',')
            path_list.append(parts[1])
            label_list.append(np.array(parts[2], dtype=np.int64))
            line = fin.readline()
            ind += 1
            if isdemo:
                if ind > 1000:
                    return path_list, label_list
    N = len(path_list)
    tr, va = int(N * 0.8), int(N * 0.9)
    train_path, train_label = path_list[0:tr], label_list[0:tr]
    valid_path, valid_label = path_list[tr:va], label_list[tr:va]
    test_path, test_label = path_list[va:], label_list[va:]
    return train_path, train_label, valid_path, valid_label, test_path, test_label

In [3]:
trp, trl, vap, val, tep, tel = CoughVID_Lists()

In [4]:
print(len(trp))
print(len(trl))
print(len(vap))
print(len(val))
print(len(tep))
print(len(tel))

12068
12068
1508
1508
1509
1509


In [ ]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-Pytorch-KZhao/')
from torch.utils.data import Dataset
from ackit.data_utils.audio import AudioSegment

from tqdm import tqdm

In [10]:
class CoughVID_Dataset(Dataset):
    def __init__(self, path_list, label_list):
        self.path_list = path_list
        self.label_list = label_list
        self.wav_list = []
        for item in tqdm(path_list, desc="Loading"):
            self.append_wav(item)

    def __getitem__(self, ind):
        return self.wav_list[ind], self.label_list[ind]

    def __len__(self):
        return len(self.path_list)

    def append_wav(self, file_path):
        audioseg = AudioSegment.from_file(file_path)
        audioseg.vad()
        audioseg.resample(target_sample_rate=16000)
        audioseg.crop(duration=3.0, mode="train")
        self.wav_list.append(audioseg.samples)

In [13]:
cough_dataset = CoughVID_Dataset(tep, tel)

Loading:   0%|                                                                                | 0/1509 [00:00<?, ?it/s]C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-Pytorch-KZhao\ackit\data_utils\audio.py:117: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.core.load(file)  # , dtype='float32')
C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Loading:   0%|                                                                        | 1/1509 [00:00<21:30,  1.17it/s]C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-Pytorch-KZhao\ackit\data_utils\audio.py:117: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.core.load(file)  # , dt

In [15]:
from ackit.data_utils.collate_fn import collate_fn
from torch.utils.data import DataLoader
train_loader = DataLoader(cough_dataset, batch_size=32, shuffle=False,
                              collate_fn=collate_fn)
for i, (x_wav, y_label, max_len_rate) in enumerate(train_loader):
    print(x_wav.shape)
    print(y_label)
    print(max_len_rate)
    break

torch.Size([32, 48000])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0,
        0, 1, 0, 1, 0, 1, 0, 0])
tensor([1.0000, 1.0000, 1.0000, 0.8891, 0.7890, 0.7432, 0.6825, 0.6652, 0.6608,
        0.6518, 0.5723, 0.5658, 0.5497, 0.5230, 0.4582, 0.4559, 0.4406, 0.4394,
        0.4144, 0.4000, 0.3951, 0.3898, 0.3882, 0.3856, 0.2651, 0.2493, 0.2477,
        0.2063, 0.2043, 0.1739, 0.1576, 0.1470])
